In [1]:
import pandas as pd
import datetime
import numpy as np
import math

In [2]:
# Import the raw data -- Here could be different data files to iterate on -- GET THIS AS INPUT select path
Mitek = pd.read_excel('Mitek_Aine/Mitek 06.23 Master Tableau data.xlsx')

Mitek['Material Number'] = Mitek['Material Number'].astype(str).str.strip() 
# to remove leading and trailing characters, cause problems while mapping

# Import the excel sheet-parameters -- GET THIS AS INPUT

df = pd.read_excel('Mitek_Aine/MitekParameters - Updated.xlsx', header=1)
df.head()

,#,INV Loc,Franchise,Month,SEQ No,Material Number,Unnamed: 6,Item Type,PII Holding,Location Code,Mfg Sourcing Location,Country Code,Target MRC,Material Grp code,Source System Code,Material Number.1,Unnamed: 16,Implant/Instrument
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,GMED,Mitek,Current Month,-1.0,NaN,NaN,"F500, F501",GMED,NaN,"DPY010, DPY022",NaN,NaN,FG,NaN,All CIP SKUs per GMED C+1 file excluded,NaN,NaN
2,2,US,Mitek,Current Month,-1.0,NaN,NaN,NaN,US,NaN,DPY010,NaN,NaN,FG,NaN,NaN,NaN,NaN
3,3,Australia,Mitek,Current Month,-1.0,NaN,NaN,NaN,ASPAC,NaN,NaN,AU,NaN,FG,NaN,NaN,NaN,IMPLANT
4,4,New Zealand,Mitek,Current Month,-1.0,NaN,NaN,NaN,ASPAC,NaN,NaN,NZ,NaN,FG,NaN,NaN,NaN,IMPLANT


In [3]:
# ERP Variables: 
# We are defining these, because each ERP has different naming convention, because of that we need to get these as input from the user
Billing_price_file_path = 'Mitek_Aine/JDE 8.12 Mitek.xlsx'
Target_MRC_code = 4471
Target_MRC_Column_in_ERP = 'BPAN8'
Material_Number_Column_in_ERP = 'TRIM(BPLITM)'
Billing_price_ICP_Column = 'BPUPRC/10000'


In [4]:
#Parameters to filter, 'Franchise'= 'Mitek', 'Month'='current_month', 'SEQ No'?
parameter_list = ['Material Number', 'Item Type', 'PII Holding', 'Location Code',
       'Mfg Sourcing Location', 'Country Code', 'Target MRC', 'Material Grp code',
       'Source System Code', 'Implant/Instrument']

In [5]:
# BILLING PRICE
Billing_price = pd.read_excel(Billing_price_file_path)
print( Billing_price.shape)

Billing_price_for_report = Billing_price[Billing_price[Target_MRC_Column_in_ERP] == Target_MRC_code]
print( Billing_price.shape)
Billing_price[Material_Number_Column_in_ERP] = Billing_price_for_report[Material_Number_Column_in_ERP].astype(str).str.strip()# to remove leading and trailing characters
Billing_price = Billing_price[[Material_Number_Column_in_ERP, Billing_price_ICP_Column, Target_MRC_Column_in_ERP]]
print( Billing_price.shape)
Billing_price= Billing_price.drop_duplicates()
Billing_price.shape

(113385, 11)
(113385, 11)
(113385, 3)


(26292, 3)

In [6]:
# Perform the merge operation Mitek + Billing_price_file
#USE variables instead of hard coding

Mitek = pd.merge(Mitek, Billing_price, left_on= ['Material Number','Target MRC'], right_on=['TRIM(BPLITM)','BPAN8'], how='left')
#Mitek2 = Mitek2.drop_duplicates()
Mitek = Mitek.drop(['TRIM(BPLITM)', 'BPAN8'], axis=1)
Mitek = Mitek.rename(columns={'BPUPRC/10000': 'JDE Billing ICP'})

In [7]:
#  FIlterin on one country and we need further iteration to get all the countries one by one!
mitek_country = Mitek[Mitek['Country Code'] == country_code]
country_parameters = df[df['INV Loc'] == country]
print(mitek_country.shape)

NameError: name 'country_code' is not defined

In [ ]:
mitek_country.head()

In [ ]:
country = 'Singapore'
country_code = 'SG'
mitek_country = Mitek[Mitek['Country Code'] == country_code]
country_parameters = df[df['INV Loc'] == country]

def filter_dataframe(mitek_country, parameter_list, country_parameters):
    for parameter in parameter_list:
        print(parameter)
        
        if (country_parameters[parameter].isna()).any():
            filter_values = ['Nothing to filter']  
        else:
            filter_values = country_parameters[parameter].tolist()[0].split(',')
            filter_values = [element.strip() for element in filter_values]

        print(filter_values)

        if 'Nothing to filter' in filter_values:
            return mitek_country
        else:
            mitek_country = mitek_country[mitek_country[parameter].isin(filter_values)]
    
    return mitek_country

# Example usage
filtered_data = filter_dataframe(mitek_country, parameter_list, country_parameters)


In [ ]:
# FILTERING on Raw data
for parameter in parameter_list:
    print(parameter)
    if (country_parameters[parameter].isna()).any():
        filter_values = ['Nothing to filter']  
    else:
        filter_values = country_parameters[parameter].tolist()[0].split(',')
        filter_values = [element.strip() for element in filter_values]

    print(filter_values)

    if 'Nothing to filter' in filter_values:
        mitek_country
    else:
        mitek_country = mitek_country[mitek_country[parameter].isin(filter_values)]


mitek_country.shape
    
    
# This filter is not in the parameter list but in Tableau:
# Maybe we need to get this input from the user, let's discuss with the user.
# Because different reports are coming from dofferent sourcing locations.
report_df = mitek_country[mitek_country['ICP Sourcing Location'] == 'GMD001']
report_df.shape

In [ ]:
# Apply the parameters on the raw_data for a country

def filter_column(df_parameters, df_data, df_parameters_cell, df_data_column):
    filter_values = df_parameters.loc[df_parameters_cell]
    
    if pd.isnull(filter_values):
        return df_data
    else:
        return df_data[df_data_column.isin(filter_values)]



In [ ]:
# COMPARISONS , THE namings should be standardized. And we need to delete hardCoding for all comparisons!

#Comparison-1:creation of "Target ICP vs. ERP"

report_df['ERP Standard Cost in Eaches'] = report_df['ERP Standard Cost in Eaches'].astype(float)
report_df['Target ICP'] = report_df['Target ICP'].astype(float)

report_df['Target ICP vs. ERP'] = (report_df['ERP Standard Cost in Eaches'] - report_df['Target ICP']).round(2)

# Creation of "Difference Target ICP to ERP STD"
report_df['Difference Target ICP to ERP STD'] = np.where(report_df['Target ICP vs. ERP'] == 0, 'Correct',
                        np.where(report_df['Target ICP vs. ERP'] == 0.01, 'Correct',
                                 np.where(report_df['Target ICP vs. ERP'] == -0.01, 'Correct', 'Incorrect')))



In [ ]:
#Comparison-2 :Creation of "Erp Std To JDE Diff", NEED TO standardize the naming of the columns
report_df['ERP Standard Cost in Eaches'] = report_df['ERP Standard Cost in Eaches'].astype(float)
report_df['JDE Billing ICP'] = report_df['JDE Billing ICP'].astype(float)

report_df['Erp Std To JDE Diff'] = (report_df['JDE Billing ICP'] - report_df['ERP Standard Cost in Eaches']).round(2)


# Creation of "Erp Std To JDE Diff Comment"
report_df['Erp Std To JDE Diff Comment'] = np.where(report_df['Erp Std To JDE Diff'] == 0, 'Correct',
                        np.where(report_df['Erp Std To JDE Diff'] == 0.01, 'Correct',
                                 np.where(report_df['Erp Std To JDE Diff'] == -0.01, 'Correct', 'Incorrect')))


In [ ]:
#Comparison-3 : Creation of "Target ICP to JDE Diff"
report_df['Target ICP to JDE Diff'] = (report_df['Target ICP'] - report_df['JDE Billing ICP']).round(2)


# Creation of "Target ICP to JDE Diff Comment"
report_df['Target ICP to JDE Diff Comment'] = np.where(report_df['Target ICP to JDE Diff'] == 0, 'Correct',
                        np.where(report_df['Target ICP to JDE Diff'] == 0.01, 'Correct',
                                 np.where(report_df['Target ICP to JDE Diff'] == -0.01, 'Correct', 'Incorrect')))

report_df.head()


In [ ]:
report_df.info()

In [ ]:
# All about the required columns and naming convention
columns_of_report = ['Material Number','ICP Sourcing Location','Mfg Sourcing Location', 'ERP Standard Cost - Currency',
                      'ERP Standard Cost in Eaches','Location Code', 'Target ICP','Target ICP - Currency','Target MRC',
                      'ICP Sourcing Location','Gross Inventory Quantity (Subledger)','ERP Standard Cost Valuation in Local Currency (Subledger)',
                      'ICP Valuation in Invoicing Currency (Subledger)','Target ICP vs. ERP','Difference Target ICP to ERP STD',
                      'JDE Billing ICP','Erp Std To JDE Diff','Erp Std To JDE Diff Comment','Target ICP to JDE Diff',
                    'Target ICP to JDE Diff Comment']

In [ ]:
report_df2 = report_df[columns_of_report]

report_df2

In [ ]:
# PIVOT Table


# Create the pivot table without totals
pivot_table = report_df2.pivot_table(
    index='Difference Target ICP to ERP STD', 
    values=[ 'Gross Inventory Quantity (Subledger)', 'Material Number',
    'ERP Standard Cost Valuation in Local Currency (Subledger)'],
    aggfunc={
        'Gross Inventory Quantity (Subledger)': 'sum',
         'Material Number': 'count',       
        'ERP Standard Cost Valuation in Local Currency (Subledger)': 'sum'
        
    }
)

# Round the values to 0 decimal places
pivot_table = pivot_table.round()

# Calculate the total values
total_row = pd.Series({
    'Gross Inventory Quantity (Subledger)': pivot_table['Gross Inventory Quantity (Subledger)'].sum(),
    'Material Number': pivot_table['Material Number'].sum(),
    'ERP Standard Cost Valuation in Local Currency (Subledger)': pivot_table['ERP Standard Cost Valuation in Local Currency (Subledger)'].sum()
}, name='Total')

# Repeat a specific row (replace 'Specific Row' with the actual row label)
correct_row = pivot_table.loc['Correct']

# Calculate the percentage row for the specific row
percentage_correct_row = (correct_row / total_row * 100).round(2)
percentage_correct_row.name = 'Correct(%)'



# Create an empty row with empty values
empty_row = pd.Series({
    'Gross Inventory Quantity (Subledger)': '',
    'Material Number': '',
    'ERP Standard Cost Valuation in Local Currency (Subledger)': ''
}, name='')

# Repeat a specific incorrect row (replace 'Incorrect Row' with the actual label of the incorrect row)
incorrect_row = pivot_table.loc['Incorrect']

# Calculate the percentage row for the incorrect row
percentage_incorrect_row = (incorrect_row / total_row * 100).round(2)
percentage_incorrect_row.name = 'Incorrect(%)'

# Create an empty row with empty values
empty_row2 = pd.Series({
    'Gross Inventory Quantity (Subledger)': '',
    'Material Number': '',
    'ERP Standard Cost Valuation in Local Currency (Subledger)': ''
}, name='')

# Combine the data
combined_df1 = pd.concat([
    pivot_table, 
    total_row.to_frame().T, 
    empty_row.to_frame().T, 
    correct_row.to_frame().T, 
    percentage_correct_row.to_frame().T,
    empty_row.to_frame().T,
    incorrect_row.to_frame().T,
    percentage_incorrect_row.to_frame().T,
    empty_row.to_frame().T
])

# Reset index
combined_df1 = combined_df1.reset_index()

# Rename columns to match the aggregation functions
combined_df1.columns = ['Row_Labels', 'Sum ERP Standard Cost Valuation in Local Currency (Subledger)', 'Sum of Gross Inventory Quantity (Subledger)', 
                       'Count of Material Number', ]


# New header row
new_header = pd.Index(['', '', 'Target ICP vs.ERP Standard', ''], name='NewHeaderRow')

# Create a MultiIndex by combining the new header with the existing column names
multiindex = pd.MultiIndex.from_arrays([new_header, combined_df1.columns], names=('', ''))

# Set the MultiIndex as the new header
combined_df1.columns = multiindex

combined_df1






In [ ]:
# One sheet in report is PIVOT: Creating pivot table of the data

# There are 3 parts and 4 Columns for each part: 
#1) Row Labels: Correct, Incorrect, Grand Total, Correct, Correct(%), Incorrect, Incorrect(%)

##These are each part of the pivot table
# 1- Target ICP vs. ERP

# 2- Erp Std To ERP Diff

# 3- Target ICP to ERP


In [ ]:
# Another sheet in report is SUMMARY: Creating summary table of the pivot table



In [ ]:
# Variables of Output
country # For other countries we need iteration here.Here is only for Singapore
output_file = country+'.xlsx'
sheet_name = 'JDE ICPs'

In [ ]:
#Output as excel

# Create an ExcelWriter object
writer = pd.ExcelWriter(output_file)

# Write each dataframe to a separate sheet
report_df2.to_excel(writer, sheet_name='Tableau Download', index=False)
Billing_price_for_report.to_excel(writer, sheet_name='JDE ICPs', index=False)

# Save the Excel file
writer.save()
# Close the writer
writer.close()
